In [288]:
using CSV,DataFrames
df = DataFrame(CSV.read("data/gap-development.tsv",delim='\t', allowmissing=:none, normalizenames=true));
first(data,6)

,ID,Text,Pronoun,Pronoun_offset,A,A_offset,A_coref,B,B_offset,B_coref,URL
,String,String,String,Int64,String,Int64,String,String,Int64,String,String
1,development-1,"Zoe Telford -- played the police officer girlfriend of Simon, Maggie. Dumped by Simon in the final episode of series 1, after he slept with Jenny, and is not seen again. Phoebe Thomas played Cheryl Cassidy, Pauline's friend and also a year 11 pupil in Simon's class. Dumped her boyfriend following Simon's advice after he wouldn't have sex with her but later realised this was due to him catching crabs off her friend Pauline.",her,274,Cheryl Cassidy,191,TRUE,Pauline,207,FALSE,http://en.wikipedia.org/wiki/List_of_Teachers_(UK_TV_series)_characters
2,development-2,"He grew up in Evanston, Illinois the second oldest of five children including his brothers, Fred and Gordon and sisters, Marge (Peppy) and Marilyn. His high school days were spent at New Trier High School in Winnetka, Illinois. MacKenzie studied with Bernard Leach from 1949 to 1952. His simple, wheel-thrown functional pottery is heavily influenced by the oriental aesthetic of Shoji Hamada and Kanjiro Kawai.",His,284,MacKenzie,228,TRUE,Bernard Leach,251,FALSE,http://en.wikipedia.org/wiki/Warren_MacKenzie
3,development-3,"He had been reelected to Congress, but resigned in 1990 to accept a post as Ambassador to Brazil. De la Sota again ran for governor of C*rdoba in 1991. Defeated by Governor Angeloz by over 15%, this latter setback was significant because it cost De la Sota much of his support within the Justicialist Party (which was flush with victory in the 1991 mid-terms), leading to President Carlos Menem 's endorsement of a separate party list in C*rdoba for the 1993 mid-term elections, and to De la Sota's failure to regain a seat in Congress.",his,265,Angeloz,173,FALSE,De la Sota,246,TRUE,http://en.wikipedia.org/wiki/Jos%C3%A9_Manuel_de_la_Sota
4,development-4,"The current members of Crime have also performed in San Francisco under the band name ''Remote Viewers``. Strike has published two works of fiction in recent years: Ports of Hell, which is listed in the Rock and Roll Hall of Fame Library, and A Loud Humming Sound Came from Above. Rank has produced numerous films (under his real name, Henry Rosenthal) including the hit The Devil and Daniel Johnston.",his,321,Hell,174,FALSE,Henry Rosenthal,336,TRUE,http://en.wikipedia.org/wiki/Crime_(band)
5,development-5,"Her Santa Fe Opera debut in 2005 was as Nuria in the revised edition of Golijov's Ainadamar. She sang on the subsequent Deutsche Grammophon recording of the opera. For his opera Doctor Atomic, Adams rewrote the role of Kitty Oppenheimer, originally a mezzo-soprano role, for soprano voice, and Rivera sang the rewritten part of Kitty Oppenheimer at Lyric Opera of Chicago, De Nederlandse Opera, and the Metropolitan Opera., all in 2007. She has since sung several parts and roles in John Adams' works, including the soprano part in El Ni*o, and the role of Kumudha in A Flowering Tree in the Peter Sellars production at the New Crowned Hope Festival in Vienna.",She,437,Kitty Oppenheimer,219,FALSE,Rivera,294,TRUE,http://en.wikipedia.org/wiki/Jessica_Rivera
6,development-6,"Sandra Collins is an American DJ. She got her start on the West Coast of the U.S. in Phoenix, Arizona and into residencies in Los Angeles, and eventually moved towards trance. She used American producers to give herself a unique sound. Collins performed for an estimated 80,000 people on the first night of Woodstock '99, and was the first female DJ featured in the Tranceport series of influential recordings. She recently has released two CD mixes under Paul Oakenfold's Perfecto label.",She,411,Collins,236,TRUE,DJ,347,FALSE,http://en.wikipedia.org/wiki/Sandra_Collins


In [289]:
df[:section_min] = min.(df[:Pronoun_offset], df[:A_offset],df[:B_offset])
df[:Pronoun_offset2] = df[:Pronoun_offset] .+ length.(df[:Pronoun])
df[:A_offset2] = df[:A_offset] .+ length.(df[:A])
df[:B_offset2] = df[:B_offset] .+ length.(df[:B])
df[:section_max] = max.(df[:Pronoun_offset2], df[:A_offset2],df[:B_offset2])
df[:Text] = replace.(df[:Text],df[:A].=>"subjecta")
df[:Text] = replace.(df[:Text],df[:B].=>"subjectb")
df[:A_dist] = abs.(df[:Pronoun_offset] .- df[:A_offset])
df[:B_dist] = abs.(df[:Pronoun_offset] .- df[:B_offset]);
train = df[:,[:Pronoun_offset,:A_offset,:B_offset,:section_min,:Pronoun_offset2,:A_offset2,:B_offset2,:section_max,:A_dist,:B_dist]]


,Pronoun_offset,A_offset,B_offset,section_min,Pronoun_offset2,A_offset2,B_offset2,section_max,A_dist,B_dist
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,274,191,207,191,277,205,214,277,83,67
2,284,228,251,228,287,237,264,287,56,33
3,265,173,246,173,268,180,256,268,92,19
4,321,174,336,174,324,178,351,351,147,15
5,437,219,294,219,440,236,300,440,218,143
6,411,236,347,236,414,243,349,414,175,64
7,273,152,253,152,276,161,264,276,121,20
8,337,173,377,173,340,181,393,393,164,40
9,246,255,267,246,249,265,273,273,9,21


In [290]:
train_X = [ [df[i,:Pronoun_offset],df[i,:A_offset],df[i,:B_offset],df[i,:section_min],df[i,:Pronoun_offset2],df[i,:A_offset2],df[i,:B_offset2],df[i,:section_max],df[i,:A_dist],df[i,:B_dist]] for i in 1:size(df)[1] ]

2000-element Array{Array{Int64,1},1}:
 [274, 191, 207, 191, 277, 205, 214, 277, 83, 67]  
 [284, 228, 251, 228, 287, 237, 264, 287, 56, 33]  
 [265, 173, 246, 173, 268, 180, 256, 268, 92, 19]  
 [321, 174, 336, 174, 324, 178, 351, 351, 147, 15] 
 [437, 219, 294, 219, 440, 236, 300, 440, 218, 143]
 [411, 236, 347, 236, 414, 243, 349, 414, 175, 64] 
 [273, 152, 253, 152, 276, 161, 264, 276, 121, 20] 
 [337, 173, 377, 173, 340, 181, 393, 393, 164, 40] 
 [246, 255, 267, 246, 249, 265, 273, 273, 9, 21]   
 [329, 196, 226, 196, 332, 214, 242, 332, 133, 103]
 [300, 168, 212, 168, 303, 184, 223, 303, 132, 88] 
 [304, 217, 285, 217, 307, 224, 302, 307, 87, 19]  
 [293, 247, 273, 247, 296, 255, 289, 296, 46, 20]  
 ⋮                                                 
 [81, 31, 54, 31, 84, 35, 58, 84, 50, 27]          
 [414, 355, 370, 355, 417, 363, 388, 417, 59, 44]  
 [301, 264, 277, 264, 304, 272, 285, 304, 37, 24]  
 [318, 101, 128, 101, 321, 106, 132, 321, 217, 190]
 [316, 343, 372, 316, 319,

In [403]:
labels = DataFrame() 
labels[:A],labels[:B] = df[:A_coref],df[:B_coref];
labels[:Neither] = zeros(2000)
labels[:A] = map(x-> x =="TRUE" ? 1 : 2, labels[:A])
labels[:B] = map(x-> x =="TRUE" ? true : false, labels[:B])
labels[:Neither] = map(x-> x = false, labels[:Neither])

#train_Y = [ [labels[i,:A],labels[i,:B],labels[i,:Neither]] for i in 1:size(labels)[1] ]
train_Y = [ onehot(labels[i,:A], 1:3) for i in 1:size(labels)[1] ]

train_XF = [Float32.(i)./100 for i in train_X]

2000-element Array{Array{Float32,1},1}:
 [2.74, 1.91, 2.07, 1.91, 2.77, 2.05, 2.14, 2.77, 0.83, 0.67]
 [2.84, 2.28, 2.51, 2.28, 2.87, 2.37, 2.64, 2.87, 0.56, 0.33]
 [2.65, 1.73, 2.46, 1.73, 2.68, 1.8, 2.56, 2.68, 0.92, 0.19] 
 [3.21, 1.74, 3.36, 1.74, 3.24, 1.78, 3.51, 3.51, 1.47, 0.15]
 [4.37, 2.19, 2.94, 2.19, 4.4, 2.36, 3.0, 4.4, 2.18, 1.43]   
 [4.11, 2.36, 3.47, 2.36, 4.14, 2.43, 3.49, 4.14, 1.75, 0.64]
 [2.73, 1.52, 2.53, 1.52, 2.76, 1.61, 2.64, 2.76, 1.21, 0.2] 
 [3.37, 1.73, 3.77, 1.73, 3.4, 1.81, 3.93, 3.93, 1.64, 0.4]  
 [2.46, 2.55, 2.67, 2.46, 2.49, 2.65, 2.73, 2.73, 0.09, 0.21]
 [3.29, 1.96, 2.26, 1.96, 3.32, 2.14, 2.42, 3.32, 1.33, 1.03]
 [3.0, 1.68, 2.12, 1.68, 3.03, 1.84, 2.23, 3.03, 1.32, 0.88] 
 [3.04, 2.17, 2.85, 2.17, 3.07, 2.24, 3.02, 3.07, 0.87, 0.19]
 [2.93, 2.47, 2.73, 2.47, 2.96, 2.55, 2.89, 2.96, 0.46, 0.2] 
 ⋮                                                           
 [0.81, 0.31, 0.54, 0.31, 0.84, 0.35, 0.58, 0.84, 0.5, 0.27] 
 [4.14, 3.55, 3.7, 3.55, 4.17,

In [416]:
#Pkg.add("NNlib")
using Flux;
using Flux:crossentropy,throttle
using StatsBase


model2 = Chain(Dense(10,5,relu),
    Dense(5,3,NNlib.σ),softmax)


    
function loss(x,y)
    x = train_XF
    y = train_Y
    i = sample(1:2000, 1, replace = false)[1]
    l = Flux.mse(model2(x[i]),y[i])
    l
end

loss (generic function with 1 method)

In [404]:
data_XY = zip(train_XF,train_Y)

Base.Iterators.Zip2{Array{Array{Float32,1},1},Array{Flux.OneHotVector,1}}(Array{Float32,1}[[2.74, 1.91, 2.07, 1.91, 2.77, 2.05, 2.14, 2.77, 0.83, 0.67], [2.84, 2.28, 2.51, 2.28, 2.87, 2.37, 2.64, 2.87, 0.56, 0.33], [2.65, 1.73, 2.46, 1.73, 2.68, 1.8, 2.56, 2.68, 0.92, 0.19], [3.21, 1.74, 3.36, 1.74, 3.24, 1.78, 3.51, 3.51, 1.47, 0.15], [4.37, 2.19, 2.94, 2.19, 4.4, 2.36, 3.0, 4.4, 2.18, 1.43], [4.11, 2.36, 3.47, 2.36, 4.14, 2.43, 3.49, 4.14, 1.75, 0.64], [2.73, 1.52, 2.53, 1.52, 2.76, 1.61, 2.64, 2.76, 1.21, 0.2], [3.37, 1.73, 3.77, 1.73, 3.4, 1.81, 3.93, 3.93, 1.64, 0.4], [2.46, 2.55, 2.67, 2.46, 2.49, 2.65, 2.73, 2.73, 0.09, 0.21], [3.29, 1.96, 2.26, 1.96, 3.32, 2.14, 2.42, 3.32, 1.33, 1.03]  …  [3.01, 2.64, 2.77, 2.64, 3.04, 2.72, 2.85, 3.04, 0.37, 0.24], [3.18, 1.01, 1.28, 1.01, 3.21, 1.06, 1.32, 3.21, 2.17, 1.9], [3.16, 3.43, 3.72, 3.16, 3.19, 3.48, 3.76, 3.76, 0.27, 0.56], [4.07, 3.63, 3.84, 3.63, 4.1, 3.77, 3.96, 4.1, 0.44, 0.23], [2.12, 1.31, 1.47, 1.31, 2.15, 1.41, 1.62, 2.15,

In [304]:
summary(model)

"Chain{Tuple{Dense{typeof(σ),TrackedArray{…,Array{Float32,2}},TrackedArray{…,Array{Float32,1}}},Dense{typeof(σ),TrackedArray{…,Array{Float32,2}},TrackedArray{…,Array{Float32,1}}},typeof(softmax)}}"

In [349]:
evalcb = () -> @show loss(train_X,train_Y)

#305 (generic function with 1 method)

In [417]:
for i  in 1:100
Flux.train!(loss, params(model2),data_XY, Descent(0.05),cb = throttle(evalcb, 30))
end

loss(train_X, train_Y) = 0.30350205f0 (tracked)
loss(train_X, train_Y) = 0.21080504f0 (tracked)
loss(train_X, train_Y) = 0.18065402f0 (tracked)
loss(train_X, train_Y) = 0.16164085f0 (tracked)
loss(train_X, train_Y) = 0.17522345f0 (tracked)
loss(train_X, train_Y) = 0.12552236f0 (tracked)
loss(train_X, train_Y) = 0.18585739f0 (tracked)
loss(train_X, train_Y) = 0.17699604f0 (tracked)
loss(train_X, train_Y) = 0.21840727f0 (tracked)
loss(train_X, train_Y) = 0.169492f0 (tracked)
loss(train_X, train_Y) = 0.17401221f0 (tracked)
loss(train_X, train_Y) = 0.16458485f0 (tracked)
loss(train_X, train_Y) = 0.15969728f0 (tracked)
loss(train_X, train_Y) = 0.17121929f0 (tracked)
loss(train_X, train_Y) = 0.28681237f0 (tracked)
loss(train_X, train_Y) = 0.27232027f0 (tracked)
loss(train_X, train_Y) = 0.21466964f0 (tracked)
loss(train_X, train_Y) = 0.19376385f0 (tracked)
loss(train_X, train_Y) = 0.25468892f0 (tracked)
loss(train_X, train_Y) = 0.16486222f0 (tracked)
loss(train_X, train_Y) = 0.14651164f0 (tra

In [420]:
model2(train_X[23])

Tracked 3-element Array{Float32,1}:
 0.42231882f0
 0.42231882f0
 0.15536241f0

In [327]:
train_Y[1656]

3-element Flux.OneHotVector:
 false
  true
 false

In [409]:
params(model2)

Params([Float32[2.66272 1.38406 … 3.3146 2.88083; 0.111684 -0.502158 … -0.893697 0.411466; … ; 0.537957 -0.414714 … -0.282643 -0.476555; -0.737402 -0.535208 … -0.910332 -0.169445] (tracked), Float32[2.50304, -0.520363, -0.0931673, 0.0, -0.429112] (tracked), Float32[2.17758 1.64441 … -0.744242 0.396827; 1.86046 0.86408 … 0.209833 -0.488413; -2.36168 -2.03426 … -0.305097 -0.373036] (tracked), Float32[1.4051, 2.20709, -1.62545] (tracked)])